In [151]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import plotly.express as px
import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm

In [152]:

wr.config.s3_endpoint_url = "http://192.168.1.7:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
]
allInputFeature = caseFeatures + allVisionFeatures
targetCol = [
    x for x in multilabelDf.columns if x not in allInputFeature and x != "CaseID"
]
saloonPredDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/saloon_v2/Saloon - 4 Dr_imgs_pred_output.csv")
hatchBackDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/hatchback_v1/Hatchback - 5 Dr_imgs_pred_output.csv")
imgPredOutput = pd.concat([saloonPredDf, hatchBackDf])
# imgPredOutput = saloonPredDf
allPredModel = []
allPreds = []
allGt = []
realTestDataDf = multilabelDf[caseFeatures + targetCol + ["CaseID"]].merge(imgPredOutput, on="CaseID")
realTestDataDf = realTestDataDf.loc[:, ~realTestDataDf.columns.str.contains('^Unnamed')]
trainDf = multilabelDf[~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())]
assert set(realTestDataDf['CaseID'].tolist()).isdisjoint(trainDf['CaseID'].tolist())

for part in tqdm(targetCol):
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []
    trainCaseId = trainDf["CaseID"].tolist()
    testCaseId = realTestDataDf["CaseID"].tolist()
    X_train = trainDf[allInputFeature]
    Y_train = trainDf[part].to_frame()
    X_test = realTestDataDf[allInputFeature]
    Y_test = realTestDataDf[part].to_frame()

    train_pool = Pool(
        X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
    )
    test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
    clf = CatBoostClassifier(
        loss_function="CrossEntropy",
        eval_metric="F1",
        iterations=20,
        task_type="GPU",
        # class_names=["not_dmg", "dmg"],
    )
    clf.fit(
        train_pool, eval_set=test_pool, metric_period=20, plot=False, verbose=0
    )
    test_predict = clf.predict(X_test)
    acc = accuracy_score(Y_test.values.astype(np.int64), test_predict)
    confMat = confusion_matrix(Y_test.values.astype(np.int64), test_predict)
    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    tn = confMat[0][0]
    tp = confMat[1][1]
    fp = confMat[0][1]
    fn = confMat[1][0]
    totalSample = fp + fn + tp + tn
    acc = (tp + tn) / (fp + fn + tp + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
        fp / (tn + fp)
    ) 
    

    allPredModel.append(
        { 
            "part": part,
            "tp": tp / (tp + fn),
            "tn": tn / (tn + fp),
            "fp": fp / (tn + fp),
            "fn": fn / (tp + fn),
            "acc": acc,
            "precision": precision,
            "recall": recall,
            "f1": (2 * precision * recall) / (precision + recall),
            "pos_count": pos_count,
        }
    )
    assert len(testCaseId) == len(test_predict)
    assert len(testCaseId) == len(Y_test)

    allPreds.append({
    "CaseID": testCaseId,
    part : test_predict})
    allGt.append({
    "CaseID": testCaseId,
    part : Y_test.values.squeeze(1)})
 

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")

print(f"avgAcc : {avgAcc}")

100%|██████████| 34/34 [00:17<00:00,  1.92it/s]

Avg Precision : 0.4988510756787962
Avg Recall : 0.661100271647091
Avg F1 : 0.5396590941794641
Avg TP : 0.661100271647091
Avg TN : 0.7308495807294422
Avg FN : 0.33889972835290905
avgAcc : 0.7825986928104575


In [153]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [154]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [155]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgAcc, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [156]:
fig = px.bar(evalMetrics, x = 'part', y = 'precision')

fig.add_hline(y=avgPrecision, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [157]:
fig = px.bar(evalMetrics, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [158]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 33/33 [00:02<00:00, 13.46it/s]


,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
40956,10000008,1,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2690,10000016,1,1,0,0,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2691,10000038,1,1,1,0,1,1,0,0,1,...,0,1,1,1,1,0,0,0,0,1
2692,10000063,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
43301,10000072,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11155,13560328,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
27954,13560507,1,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
31023,13560636,0,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31024,13560974,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)

100%|██████████| 33/33 [00:02<00:00, 13.29it/s]


In [160]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
40956,10000008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2690,10000016,1,1,1,1,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2691,10000038,1,1,1,0,1,1,0,0,1,...,1,1,0,1,1,0,0,0,0,0
2692,10000063,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
43301,10000072,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11155,13560328,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27954,13560507,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
31023,13560636,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
31024,13560974,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [161]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list), acc_list

In [162]:
subsetAcc2, acc_list = hamming_score(multilabelGt[targetCol].values, multilabelPred[targetCol].values)
subsetAcc2

0.7658514545132196